# Simulating Asset Price Evolutions and Reprice Risky up-and-out Call Option

## Introduction

The model we implement in this assignment is the LIBOR forward rate model to simulate interest rates. The initial values for the LIBOR forward rates need to be calibrated to the market forward rates which can be deduced through the market zero-coupon bond prices. This continuously compounded interest rate is given by,

$$e^{r_{ti}(t_{i+1}-t_{i})} = 1 + L(t_{i},t_{i+1})(t_{i+1}-t_{i})$$

We initialize most variables as given by the question.

- Option maturity is one year <br>
- The option is struck at-the-money <br>
- The up-and-out barrier for the option is USD 150<br>
- The current share price is USD 100<br>
- The current firm value for the counterparty is USD 200<br>
- The counterparty’s debt, due in one year, is USD 175<br>
- The correlation between the counterparty and the stock is constant at 0.2 <br>
- The recovery rate with the counterparty is 25% 

## 1. LIBOR Forward Rates, Stock Paths, and Counterparty Firm Values

In this part, we use a sample size of 100000, jointly simulate LIBOR forward rates, stock paths, and counterparty firm values. We assume that the counterparty firm and stock values are uncorrelated with LIBOR forward rates.

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import norm
import scipy.optimize

import math
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
### Initialize problem parameters
T = 1 # option maturity
L = 150 # up-and-out barrier
S0 = 100 # current share price
K = 100 # strike price, at-the-money


v_0 = 200 # counterparty firm current value
debt = 175 # counterparty's debt, due in one year 
corr = .2 # correlation
recovery_rate = 0.25 # recovery rate
########

corr_matrix = np.array([[1, corr], [corr, 1]])
sample_size = 100000

In [ ]:
sigma_const = 0.30
gamma = 0.75

### Calibrate LIBOR forward rate model from zero coupon bond prices

We initialize the given zero-coupon bond prices:

In [ ]:
# which gives monthly time intervals
t = np.linspace(0,1,13) 

market_zcb_prices = np.array([1.0, 0.9938, 0.9876, 0.9815, 0.9754, 0.9694, 0.9634, 0.9574, 0.9516,
       0.9457, 0.9399, 0.9342, 0.9285])

We next create functions to calculate the simulated bond prices from the Vasicek model (as well as helper functions A and D). We also define function F which is the differences between the bond prices calculated by our model and actual market zero-coupon bond prices :

In [ ]:
def A(t1, t2, alpha):
    return (1 - np.exp(-alpha*(t2-t1)))/alpha

def D(t1, t2, alpha, b, sigma):
    val1 = (t2 - t1 - A(t1,t2,alpha)) * (sigma**2/(2 * alpha**2) - b)
    val2 = sigma**2 * A(t1,t2,alpha)**2 / (4*alpha)
    return val1-val2

def bond_price_fun(r,t,T, alpha, b, sigma):
    return np.exp(-A(t,T,alpha)*r + D(t,T,alpha,b,sigma))

def F(x):
    alpha = x[0]
    b = x[1]
    sigma = x[2]
    r0 = x[3]
    return sum( np.abs(bond_price_fun(r0,0,t,alpha,b,sigma) - market_zcb_prices))

scipy library provides fmin_slsqp method which looks for min value of provide function with Sequential Least Squares Programming method. In our case that would bring optimal model parameters which simulated market prices.

In [ ]:
#minimizing F
bnds = ((0,1),(0,0.2),(0,0.2), (0.00,0.10))
opt_val = scipy.optimize.fmin_slsqp(F, (0.3, 0.05, 0.03, 0.05), bounds=bnds)
opt_alpha = opt_val[0]
opt_b = opt_val[1]
opt_sigma = opt_val[2]
opt_r0 = opt_val[3]

fmin_slsqp - exit mode is 0 , which means optimization has found a solution.

In [ ]:
print("Optimal alpha: :%.3f" % opt_alpha)
print("Optimal b: %.3f" % opt_b)
print("Optimal sigma %.3f" % opt_sigma )
print("Optimal r0: %.3f" % opt_r0 )

We plot the actual market bond prices, with model-derived bond prices, and they look like a close fit.

In [ ]:
%%capture output 
model_prices = bond_price_fun(opt_r0,0,t, opt_alpha, opt_b, opt_sigma)
model_yield = -np.log(model_prices) / t

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))
ax.plot(t, market_zcb_prices, label='Actual Market Prices')
ax.plot(t, model_prices, 'x', label='Vasicek Model prices')
plt.xlabel("Maturity")
plt.ylabel("Bond Price")
plt.legend()
plt.show()

We calculate forward rates based on prices of zero-coupon bonds.

### Simulate LIBOR rate paths

We first initialize the parameter <math xmlns="http://www.w3.org/1998/Math/MathML">
  <msub>
    <mi>&#x3C3;</mi>
    <mi>j</mi>
  </msub>
</math>

In [ ]:
sigmaj = 0.2

We use paramters we obtained above to recreate the Vasicek bond prices:

In [ ]:
def A(t1, t2):
    return (1 - np.exp(-opt_alpha * (t2-t1)))/opt_alpha

def C(t1, t2):
    val1 = (t2 - t1 - A(t1,t2)) * (opt_sigma**2/ (2 * opt_alpha**2) - opt_b)
    val2 = opt_sigma**2 * A(t1,t2)**2 / (4 * opt_alpha)
    return val1 - val2

def bond_price(r,t,T):
    return np.exp( -A(t,T)*r + C(t,T))

vasi_bond = bond_price(opt_r0, 0, t)

The prices calculated from the Vasicek model are close to the ZCB prices given by the assignment as shown below :

In [ ]:
print("Model zero coupon bond prices: ", vasi_bond)
print("Market zero coupon bond prices: ", market_zcb_prices)

We now initialize the matrices we will use to store the Monte Carlo simulations, for both basic Monte Carlo and Predictor-Corrector method.

In [ ]:
n_simulations = sample_size
n_steps = len(t)

mc_forward = np.ones([n_simulations, n_steps-1])*(vasi_bond[:-1]-vasi_bond[1:])/(vasi_bond[1:])
predcorr_forward = np.ones([n_simulations, n_steps-1])*(vasi_bond[:-1]-vasi_bond[1:])/(vasi_bond[1:])
predcorr_capfac = np.ones([n_simulations, n_steps])
mc_capfac = np.ones([n_simulations, n_steps])

delta = np.ones([n_simulations, n_steps - 1])*(t[1:]-t[:-1])

We now run the Monte Carlo simulation for each time step:

In [ ]:
for i in range(1, n_steps):
    Z = norm.rvs(size=[n_simulations,1])
    
    muhat = np.cumsum(delta[:, i:] * mc_forward[:, i:] * sigmaj**2 \
                       /(1 + delta[:, i:] * mc_forward[:,i:]), axis=1)
    
    mc_forward[:,i:] = mc_forward[:,i:] * \
                            np.exp((muhat-sigmaj**2/2)*delta[:,i:]
                                       +sigmaj*np.sqrt(delta[:,i:]) * Z)
    
    mu_initial = np.cumsum(delta[:,i:]*predcorr_forward[:,i:]*sigmaj**2 \
                           /(1 + delta[:,i:] * predcorr_forward[:,i:]), axis=1)
    
    for_temp = predcorr_forward[:,i:] * \
                    np.exp((mu_initial - sigmaj**2 / 2) * delta[:,i:] \
                                  + sigmaj * np.sqrt(delta[:,i:]) * Z)
    
    mu_term = np.cumsum(delta[:,i:] * for_temp*sigmaj**2 \
                         /(1 + delta[:,i:] * for_temp), axis=1)
    
    predcorr_forward[:,i:] = predcorr_forward[:,i:] * \
                                np.exp((mu_initial + mu_term - sigmaj**2) * delta[:,i:]/2 \
                                                             + sigmaj * np.sqrt(delta[:,i:]) * Z)

Capitalization factors and bond prices are simulated using Monte Carlo simulation, later we plot them to compare with the Vasicek bond prices.



In [ ]:
mc_capfac[:,1:] = np.cumprod(1 + mc_forward, axis=1)
predcorr_capfac[:,1:] = np.cumprod(1 + predcorr_forward, axis=1)

mc_price = mc_capfac**(-1)
predcorr_price = predcorr_capfac**(-1)

mc_final = np.mean(mc_price, axis=0)
predcorr_final = np.mean(predcorr_price, axis=0)
predcorr_final

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))
ax.plot(t,vasi_bond, label="Vasicek Bond Prices")
ax.plot(t, mc_final, 'o', label="Simple Monte Carlo Bond Prices")
ax.plot(t, predcorr_final, 'x', label="Predictor-Corrector Bond Prices")

plt.xlabel("Maturity")
plt.ylabel("Bond Price")
plt.legend()
plt.show()

In [ ]:
model_forward_rates = (vasi_bond[:-1]-vasi_bond[1:])/ vasi_bond[1:]
pred_corr_forward_rates = np.mean(predcorr_forward, axis=0)

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))
ax.plot(t[1:], pred_corr_forward_rates * 100, label='Vasicek model forward rates(1m)')
ax.plot(t[1:], model_forward_rates * 100, 'x', label='Pred-corr forward rates(1m)')

plt.xlabel("Maturity")
plt.ylabel("Fwd rates %")
plt.legend()
plt.show()

We calculate continuous compounded interest rates using Predictor-Corrector method from simulated forward rates

$$e^{r_{ti}(t_{i+1}-t_{i})} = 1 + L(t_{i},t_{i+1})(t_{i+1}-t_{i})$$

In [ ]:
r_sim = np.log(1 + predcorr_forward*(delta))/delta
r_sim

In [ ]:
r_sim_annualized = pd.DataFrame(r_sim/delta)
r_sim_annualized

## Generate stock and firm values

Cholesky decomposition will provide correlate price paths with predifined correlation factor.

In [ ]:
def next_share_price(prev_price, r, dT, sigma_const, gamma, sample_size, Z, varying_vol = True):

    if varying_vol:
        sigma = sigma_const*(prev_price)**(gamma-1)
    else:
        sigma = sigma_const*(S0)**(gamma-1)
    
    return prev_price*np.exp(np.cumsum((r-(sigma**2)/2)*(dT)+(sigma)*(np.sqrt(dT))*Z,1))

def generate_share_and_firm_price(S0, v_0, r_sim, sigma_const, gamma, corr, T, sample_size, timesteps = 12):
    corr_matrix = np.array([[1, corr], [corr, 1]])
    norm_matrix = stats.norm.rvs(size = np.array([sample_size, 2, timesteps]))
    corr_norm_matrix = np.matmul(np.linalg.cholesky(corr_matrix), norm_matrix)
    
    
    share_price_path = pd.DataFrame(
         next_share_price(S0, r_sim, 1/timesteps, sigma_const, gamma, sample_size, Z=corr_norm_matrix[:,0,]))
    share_price_path = share_price_path.transpose()
    
    first_row = pd.DataFrame([S0]*sample_size)
    first_row = first_row.transpose()
    share_price_path = pd.concat([first_row, share_price_path])
    share_price_path = share_price_path.reset_index(drop=True)

    firm_price_path = pd.DataFrame(
        next_share_price(v_0, r_sim, 1/timesteps, sigma_const, gamma, sample_size, Z=corr_norm_matrix[:,1,]))
    firm_price_path = firm_price_path.transpose()
    
    first_row = pd.DataFrame([v_0]*sample_size)
    first_row = first_row.transpose()
    firm_price_path = pd.concat([first_row, firm_price_path])
    firm_price_path = firm_price_path.reset_index(drop=True)

    return [share_price_path,firm_price_path]  

In [ ]:
share_prices, firm_prices = generate_share_and_firm_price(S0, v_0, 
                                                          r_sim_annualized, 
                                                          sigma_const, 
                                                          gamma, 
                                                          corr, T, sample_size, timesteps = 12)

We then print out the share price and firm value paths:

In [ ]:
share_prices

In [ ]:
firm_prices

We also plot the first 1000 stock price and firm value paths simulated:

In [ ]:
share_prices.iloc[:,0:1000].plot(
    title='Share price over 12 months', 
    legend=False,
    figsize=(12, 4));

In [ ]:
firm_prices.iloc[:,0:1000].plot(
    title='Firm price over 12 months', 
    legend=False,
    figsize=(12, 4));

# 2. Discount Factor and Value of the Up-and-Out Call Option

In this section we use the capitalisation factor calculated in the section above to obtain one-year discount factor. For each simulation - we build firm stock and firm price paths and resolve option payoff to get the final price.

We first calculated the one year discount factor, by inverting the capitalisation factor. The capitalisation factor is calculated by taking the cumulative product of the interest rate between each timestep.

In [ ]:
one_year_disc_fac = 1/np.cumprod(1+r_sim,1)[:,-1]
print(one_year_disc_fac)

Next, we calculate the default-free option value. \ One difference is that we multiple the payoff by the one year discount factor, instead of multiplying with  $e^{rT}$

In [ ]:
# define payoff for up-and-out call option
def payoff(S_t, K, L):
    stopped_S = S_t.iloc[-1].where((S_t < L).all(), 0)
    return np.maximum(stopped_S - K, 0).to_numpy()

In [ ]:
# Estimate the default-free value of the option:
option_estimate = []
option_std = []

payoffs = payoff(share_prices, K, L)
option_price = one_year_disc_fac*payoffs
print("Simulated option prices: %s" % option_price)
print("Simulated payoffs : %s" % payoffs)

In [ ]:
option_estimate = option_price.mean()
option_std = option_price.std()/np.sqrt(sample_size)

print("default-free option price: %.3f" % option_estimate)
print("default-free option price std: %.3f" % option_std)

Next, we incorporate the CVA Adjustment similar to the first submission.

In [ ]:
payoffs = payoff(share_prices, K, L)
term_firm_vals = firm_prices.iloc[-1].to_numpy()
amount_lost = one_year_disc_fac*(1-recovery_rate)*(term_firm_vals < debt)*payoffs
cva_estimate = amount_lost.mean()
cva_std = amount_lost.std()/np.sqrt(sample_size)

option_cva_price = option_price - amount_lost
option_cva_adjusted_prices = option_cva_price.mean()
option_cva_adjusted_std = option_cva_price.std()/np.sqrt(sample_size)

In [ ]:
print("Credit value adjustment: %.3f" % cva_estimate)
print("Credit value adjustment std: %.3f" % cva_std)
print("CVA-adjusted option price: %.3f" % option_cva_adjusted_prices)
print("CVA-adjusted option price std: %.3f" % option_cva_adjusted_std)

In [ ]:
option_cva_adjusted_prices = []
option_cva_adjusted_std = []

for sample_size, paths in share_price_paths.items(): 
    payoffs = payoff(paths, K, L)
    option_price = np.exp(-risk_free*T)*payoffs

    term_firm_vals = firm_val_paths[sample_size].iloc[-1].to_numpy()
    amount_lost = np.exp(-risk_free*T)*(1-recovery_rate)*(term_firm_vals < debt)*payoffs
    
    option_cva_price = option_price - amount_lost
    
    option_cva_adjusted_prices.append(option_cva_price.mean())
    option_cva_adjusted_std.append(option_cva_price.std()/np.sqrt(sample_size))